In [3]:
%cd ..

/Users/georgi/dev/dialogue_modeling


In [7]:
import spacy
import string
from read_data import read_solution_annotaions, read_wason_dump, read_3_lvl_annotation_file
from wason_message import WasonConversation, WasonMessage
import pandas as pd
from solution_tracker.simple_sol import process_raw_to_solution_tracker

In [11]:
from collections import defaultdict

In [14]:
anns = read_solution_annotaions('solution_annotations.tsv')
nlp = spacy.load("en_core_web_sm")
for a in anns:
    a.preprocess_everything(nlp)

raw_data = read_wason_dump('data/all/')

conversations_to_process = []
for ann in anns:
    raw = [r for r in raw_data if r.identifier == ann.identifier][0]
    ann.raw_db_conversation = raw.raw_db_conversation
    conversations_to_process.append(ann)

hierch_data = read_3_lvl_annotation_file('3lvl_anns.tsv')

for conv in anns:
    hierch = [d for d in hierch_data if d.identifier == conv.identifier][0]
    conv.merge_all_annotations(hierch)

correct = 0
total = 0
p_correct = 0
p_total = 0

sols = defaultdict(lambda x: [])
for conv in conversations_to_process:
    c_correct, c_total, cp_correct, cp_total, sol_tracker = process_raw_to_solution_tracker(conv)
    correct += c_correct
    total += c_total
    p_correct += cp_correct
    p_total += cp_total
    sols[conv.identifier] = sol_tracker

print(correct/total)
print(p_correct/p_total)

0.7634408602150538
0.7530864197530864


In [64]:
import copy
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', 0)

In [65]:
for k_sol, sol in sols.items():
    
    wason_conv = [a for a in anns if a.identifier == k_sol][0]
    latest_sol = {}
    
    for item in sol:
        if item['type'] == 'INITIAL':
            latest_sol[item['user']] = " ".join(item['value'])
        else: 
            if item['user'] not in latest_sol:
                latest_sol[item['user']] = 'N/A'
            
    df = pd.DataFrame(columns=['origin', 'content', *latest_sol.keys()])        
    
    
    df = df.append({'origin': 'initial', 'content': 'Solo Submissions', **latest_sol},
                   ignore_index=True)
    
    for raw in wason_conv.raw_db_conversation:
        if raw['user_status'] != 'USR_PLAYING':
            continue
            
        if raw['message_type'] == 'WASON_SUBMIT':
            local_sol = [s for s in sol if s['id'] == raw['message_id']][0]
            norm_value = " ".join(local_sol['value'])
            
            if norm_value == latest_sol[local_sol['user']]:
                display_value = norm_value
            else:
                display_value = "*** {} ***".format(norm_value)
                
            latest_sol[local_sol['user']] = norm_value
            
            display_dict = copy.copy(latest_sol)
            display_dict[local_sol['user']] = display_value
            
            df = df.append({'origin': local_sol['user'], 'content': 'Submission', **display_dict},
               ignore_index=True)
            
        elif raw['message_type'] == 'CHAT_MESSAGE':
            local_sol = [s for s in sol if s['id'] == raw['message_id']]
            if len(local_sol) == 0:
                df = df.append({'origin': raw['user_name'], 'content': raw['content'], **latest_sol},
               ignore_index=True)
            else:
                local_sol = local_sol[0]
                norm_value = " ".join(local_sol['value'])
            
                if norm_value == latest_sol[local_sol['user']]:
                    display_value = norm_value
                else:
                    display_value = "*** {} ***".format(norm_value)

                latest_sol[local_sol['user']] = norm_value

                display_dict = copy.copy(latest_sol)
                display_dict[local_sol['user']] = display_value

                df = df.append({'origin': local_sol['user'], 'content': raw['content'], **display_dict},
                   ignore_index=True)
                

    h = HTML(df.to_html())
    display(h)

,origin,content,Alpaca,Leopard,Chipmunk
0,initial,Solo Submissions,3 U,U 4,3
1,Leopard,hi,3 U,U 4,3
2,Chipmunk,Hello,3 U,U 4,3
3,Alpaca,hiya,3 U,U 4,3
4,Leopard,is it 4 and u?,3 U,U 4,3
5,Chipmunk,So what do you guys think?,3 U,U 4,3
6,Chipmunk,I think we only need to do 2 right?,3 U,U 4,3
7,Alpaca,I believe the answer is 3 and u.,3 U,U 4,3
8,Leopard,I don't have a 2,3 U,U 4,3
9,Chipmunk,No I meant like two cards,3 U,U 4,3


,origin,content,Beaver,Duck,Ox
0,initial,Solo Submissions,I 4,9 4 J,I 4
1,Ox,Hello,I 4,9 4 J,I 4
2,Duck,Hi,I 4,9 4 J,I 4
3,Beaver,Hello,I 4,9 4 J,I 4
4,Beaver,What do you guys think is the best solution?,I 4,9 4 J,I 4
5,Duck,my guess is all of them except 1,I 4,*** I 9 4 J ***,I 4
6,Ox,"Did you pick I and 4 as your cards, I did. They would be the only ones you would have to check to verify the rule, right?",I 4,I 9 4 J,I 4
7,Beaver,Why do you think that @Duck?,I 4,I 9 4 J,I 4
8,Duck,All cards with vowels on one side have an even number on the other,I 4,I 9 4 J,I 4
9,Duck,it's already given,I 4,I 9 4 J,I 4


,origin,content,Chipmunk,Duck,Zebra,Beaver
0,initial,Solo Submissions,2 U,B 2 U 7,U 7,B 2 U 7
1,Chipmunk,Submission,2 U,B 2 U 7,U 7,B 2 U 7
2,Duck,Submission,2 U,*** U ***,U 7,B 2 U 7
3,Zebra,What answers did everyone put?,2 U,U,U 7,B 2 U 7
4,Duck,Okay so what answer did everyone put to see if we all put the same answer?,2 U,U,U 7,B 2 U 7
5,Zebra,I put 7 and U,2 U,U,U 7,B 2 U 7
6,Beaver,I put U and 2,2 U,U,U 7,*** 2 U ***
7,Chipmunk,I selected U and 2 as they are vowels and even number,2 U,U,U 7,2 U
8,Duck,Submission,2 U,*** 2 U ***,U 7,2 U
9,Zebra,I think it would be 7 to test if it would be incorrect,2 U,2 U,*** 7 ***,2 U


,origin,content,Zebra,Duck,Cat
0,initial,Solo Submissions,2 A,5 C 2 A,2
1,Zebra,Hi,2 A,5 C 2 A,2
2,Duck,Hi Zebra,2 A,5 C 2 A,2
3,Zebra,What did you press,2 A,5 C 2 A,2
4,Cat,Hi,2 A,5 C 2 A,2
5,Duck,Cat looks foxy,2 A,5 C 2 A,2
6,Cat,I pressed 2,2 A,5 C 2 A,2
7,Duck,I clicked all of the cards,2 A,5 C 2 A,2
8,Zebra,I did 2 and A,2 A,5 C 2 A,2
9,Duck,Why 2 & a,2 A,5 C 2 A,2


,origin,content,Guinea pig,Tiger,Bee,Llama,Ox
0,initial,Solo Submissions,5 2,5 U,N/A,N/A,N/A
1,Bee,hi,5 2,5 U,N/A,N/A,N/A
2,Guinea pig,hi,5 2,5 U,N/A,N/A,N/A
3,Bee,Submission,5 2,5 U,*** U ***,N/A,N/A
4,Llama,hi,5 2,5 U,U,N/A,N/A
5,Guinea pig,Submission,*** U ***,5 U,U,N/A,N/A
6,Llama,Submission,U,5 U,U,*** U ***,N/A
7,Llama,Submission,U,5 U,U,*** 2 U ***,N/A
8,Bee,U is the only vowel there so we must turn it to see whether it has an even number on the other side,U,5 U,U,2 U,N/A
9,Llama,yes,U,5 U,U,*** U ***,N/A


...

,origin,content,Falcon,Lion,Chipmunk
0,initial,Solo Submissions,8,N/A,N/A
1,Falcon,its either 8 or O,8,N/A,N/A
2,Falcon,or we just both,8,N/A,N/A
3,Falcon,*we just use,8,N/A,N/A
4,Falcon,Submission,8,N/A,N/A
5,Lion,I think both,8,N/A,N/A
6,Falcon,Submission,*** 8 O ***,N/A,N/A
7,Lion,8 and O,8 O,*** 8 O ***,N/A
8,Falcon,ok,8 O,8 O,N/A
9,Falcon,that was quick,8 O,8 O,N/A


,origin,content,Tiger,Kiwi,Beaver,Lion
0,initial,Solo Submissions,A 6,N/A,N/A,N/A
1,Kiwi,the obvious answer would be to chhose A and 6,A 6,*** A 6 ***,N/A,N/A
2,Kiwi,what do you think,A 6,A 6,N/A,N/A
3,Beaver,I agree. If A = even then 6 should be a vowel,A 6,A 6,N/A,N/A
4,Lion,"yes, it would I think its 3 and A because a is the vowel so we have to test if there would be an even number when turned over and 3 is an odd number so if it had a vowel the rule wouldnt be right",A 6,A 6,N/A,N/A
5,Beaver,Actually wouldn't you have to turn all of the cards?,A 6,A 6,*** 6 Z 3 A ***,N/A
6,Beaver,"to test the rule, we need to check the other cards don't follow the rule",A 6,A 6,6 Z 3 A,N/A
7,Kiwi,yeah to then see if there are any excetions to the rule,A 6,A 6,6 Z 3 A,N/A
8,Kiwi,what happens now?,A 6,A 6,6 Z 3 A,N/A
9,Lion,do we need to turn z?,A 6,A 6,6 Z 3 A,*** Z ***


,origin,content,Beaver,Lion,Giraffe
0,initial,Solo Submissions,5 X 8 E,5 8 E,N/A
1,Lion,What did you guys say was the answer?,5 X 8 E,5 8 E,N/A
2,Giraffe,5 is not an even we know tat,5 X 8 E,5 8 E,N/A
3,Giraffe,that*,5 X 8 E,5 8 E,N/A
4,Giraffe,"i put E and 5 , you?",5 X 8 E,5 8 E,*** 5 E ***
5,Lion,"E, 8 and 5",5 X 8 E,5 8 E,5 E
6,Lion,Why did you think it wasn't 8?,5 X 8 E,5 8 E,5 E
7,Beaver,i chose all 4 cards so clearly mine wasn't the one,5 X 8 E,5 8 E,5 E
8,Giraffe,Urm im thinking,5 X 8 E,5 8 E,5 E
9,Lion,"It might be right, we need to discuss",5 X 8 E,5 8 E,5 E


,origin,content,Llama,Cat,Kiwi,Bee
0,initial,Solo Submissions,E,5 E,N/A,N/A
1,Cat,i think its 5 and e,E,5 E,N/A,N/A
2,Kiwi,I also think it's 5 and E,E,5 E,*** 5 E ***,N/A
3,Cat,because the E can prove the rule and then the 5 could disprove it,E,5 E,5 E,N/A
4,Bee,How many cards can you pick?,E,5 E,5 E,N/A
5,Kiwi,You can pick 2,E,5 E,5 E,N/A
6,Kiwi,Oh,E,5 E,5 E,N/A
7,Cat,i think as many as you like,E,5 E,5 E,N/A
8,Cat,but it says must,E,5 E,5 E,N/A
9,Kiwi,Actually you could turn them all!,E,5 E,*** 5 8 G E ***,N/A


,origin,content,Lion,Ox,Bat
0,initial,Solo Submissions,U 4,9 U,U 4
1,Lion,Submission,U 4,9 U,U 4
2,Bat,Hello,U 4,9 U,U 4
3,Lion,hihi,U 4,9 U,U 4
4,Ox,Hello :),U 4,9 U,U 4
5,Bat,What do you guys think is the answer?,U 4,9 U,U 4
6,Bat,"Also just to check are the cards you can see 9,L,4,U?",U 4,9 U,U 4
7,Ox,I chose 9 and U.,U 4,9 U,U 4
8,Lion,Yes i have 9L4U too,*** 9 U ***,9 U,U 4
9,Ox,"Yes, I have those.",9 U,9 U,U 4


In [60]:
h

,origin,content,Lion,Ox,Bat
0,initial,Solo Submissions,U 4,9 U,U 4
1,Lion,Submission,U 4,9 U,U 4
2,Bat,Hello,U 4,9 U,U 4
3,Lion,hihi,U 4,9 U,U 4
4,Ox,Hello :),U 4,9 U,U 4
5,Bat,What do you guys think is the answer?,U 4,9 U,U 4
6,Bat,"Also just to check are the cards you can see 9,L,4,U?",U 4,9 U,U 4
7,Ox,I chose 9 and U.,U 4,9 U,U 4
8,Lion,Yes i have 9L4U too,*** 9 U ***,9 U,U 4
9,Ox,"Yes, I have those.",9 U,9 U,U 4


In [53]:
df

,origin,content,Lion,Ox,Bat
0,initial,Solo Submissions,U 4,9 U,U 4
1,Lion,Submission,U 4,9 U,U 4
2,Bat,Hello,U 4,9 U,U 4
3,Lion,hihi,U 4,9 U,U 4
4,Ox,Hello :),U 4,9 U,U 4
5,Bat,What do you guys think is the answer?,U 4,9 U,U 4
6,Bat,"Also just to check are the cards you can see 9,L,4,U?",U 4,9 U,U 4
7,Ox,I chose 9 and U.,U 4,9 U,U 4
8,Lion,Yes i have 9L4U too,*** 9 U ***,9 U,U 4
9,Ox,"Yes, I have those.",9 U,9 U,U 4


In [21]:
latest_sol

{'Lion': {'4', 'U'}, 'Ox': {'9', 'U'}, 'Bat': {'4', 'U'}}